# 서울시 편의점 위치와 매출의 관계 분석
## 03. 심층 분석 (회귀분석, VIF, p-value)

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from scipy import stats

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

PROJECT_DIR = r'C:/Users/Administrator/Desktop/자료모음/프로젝트_서울시_편의점_분석'
OUTPUT_DIR = f'{PROJECT_DIR}/outputs'
merged = pd.read_csv(f'{PROJECT_DIR}/통합_데이터.csv', encoding='utf-8-sig')

print(f'데이터 로드: {merged.shape}')

## 1. 분석 데이터 준비

In [ ]:
# 결측치 제거
analysis_df = merged.dropna(subset=['당월_매출_금액', '점포_수', '총_유동인구_수', '주요_상권유형']).copy()
print(f'분석 대상: {analysis_df.shape[0]:,}건 (결측치 제거 후)')

# 상권유형 더미변수 생성 (골목상권 = 기준)
analysis_df['발달상권'] = (analysis_df['주요_상권유형'] == '발달상권').astype(int)
analysis_df['전통시장'] = (analysis_df['주요_상권유형'] == '전통시장').astype(int)

print('\n더미변수 생성 완료')
print('  - 기준 범주: 골목상권')
print('  - 더미변수: 발달상권, 전통시장')

## 2. 변수 정의

In [ ]:
print('='*60)
print('변수 정의')
print('='*60)
print('\n종속변수(Y): 당월_매출_금액 (행정동별 분기 편의점 추정매출)')
print('\n독립변수(X):')
print('  1. 점포_수: 편의점 밀집도 (행정동별 분기 편의점 수)')
print('  2. 총_유동인구_수: 유동인구 (행정동별 분기 합계)')
print('  3. 발달상권: 상권유형 더미 (골목상권 대비)')
print('  4. 전통시장: 상권유형 더미 (골목상권 대비)')

X_cols = ['점포_수', '총_유동인구_수', '발달상권', '전통시장']
X = analysis_df[X_cols].astype(float)
y = analysis_df['당월_매출_금액'].astype(float)

print(f'\nX shape: {X.shape}, y shape: {y.shape}')

---
## 3. VIF (다중공선성) 분석

In [ ]:
print('='*60)
print('VIF (Variance Inflation Factor) 분석')
print('='*60)
print('\n기준:')
print('  - VIF < 5: 다중공선성 없음')
print('  - VIF 5~10: 주의 필요')
print('  - VIF > 10: 심각한 다중공선성')

# 수치형 변수만 VIF 계산
numeric_cols = ['점포_수', '총_유동인구_수']
X_numeric = X[numeric_cols].values

vif_data = pd.DataFrame()
vif_data['변수'] = numeric_cols
vif_data['VIF'] = [variance_inflation_factor(X_numeric, i) for i in range(len(numeric_cols))]

print('\n[VIF 결과]')
for idx, row in vif_data.iterrows():
    status = '다중공선성 없음' if row['VIF'] < 5 else '주의 필요' if row['VIF'] < 10 else '심각'
    print(f"  {row['변수']}: VIF = {row['VIF']:.4f} ({status})")

print('\n결론: 독립변수 간 다중공선성 문제 없음')

---
## 4. 다중 회귀분석 (OLS)

In [ ]:
# 상수항 추가 및 모델 적합
X_with_const = sm.add_constant(X)
model = sm.OLS(y, X_with_const).fit()

print('='*60)
print('다중 회귀분석 결과')
print('='*60)
print(f'\nR-squared: {model.rsquared:.4f}')
print(f'Adj. R-squared: {model.rsquared_adj:.4f}')
print(f'F-statistic: {model.fvalue:.2f}')
print(f'Prob (F-statistic): {model.f_pvalue:.2e}')
print(f'관측치 수: {int(model.nobs)}')

In [ ]:
print('='*70)
print('회귀계수 및 p-value')
print('='*70)
print(f'{"변수":<20} {"계수":>18} {"p-value":>15} {"유의성":>10}')
print('-'*70)

for var in model.params.index:
    coef = model.params[var]
    pval = model.pvalues[var]
    sig = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
    print(f'{var:<20} {coef:>18,.2f} {pval:>15.4e} {sig:>10}')

print('-'*70)
print('유의수준: *** p<0.001, ** p<0.01, * p<0.05')

In [ ]:
# statsmodels 전체 요약
print(model.summary())

---
## 5. 분석 결과 해석

In [ ]:
print('='*70)
print('분석 결과 해석')
print('='*70)

print(f'\n(1) 모델 적합도')
print(f'    - R² = {model.rsquared:.4f}')
print(f'    - 독립변수들이 매출 변동의 약 {model.rsquared*100:.1f}%를 설명')
print(f'    - F-검정 p-value < 0.001로 모델 전체가 통계적으로 유의함')

print(f'\n(2) 변수별 영향력')

coef_s = model.params['점포_수']
pval_s = model.pvalues['점포_수']
print(f'\n    [점포수 (편의점 밀집도)]')
print(f'    - 회귀계수: {coef_s:,.0f}원')
print(f'    - p-value: {pval_s:.4e}')
print(f'    - 해석: 편의점 1개 증가 시 매출 약 {coef_s/1e8:.2f}억원 증가')

coef_p = model.params['총_유동인구_수']
pval_p = model.pvalues['총_유동인구_수']
print(f'\n    [유동인구]')
print(f'    - 회귀계수: {coef_p:.4f}원')
print(f'    - p-value: {pval_p:.4e}')
print(f'    - 해석: 유동인구 100만명 증가 시 매출 약 {coef_p*1e6/1e8:.2f}억원 증가')

print(f'\n    [상권유형] (기준: 골목상권)')
coef_d = model.params['발달상권']
pval_d = model.pvalues['발달상권']
print(f'    - 발달상권: 계수 = {coef_d/1e8:.2f}억원, p-value = {pval_d:.4e}')
print(f'      → 골목상권 대비 매출이 약 {coef_d/1e8:.2f}억원 높음')

coef_t = model.params['전통시장']
pval_t = model.pvalues['전통시장']
print(f'    - 전통시장: 계수 = {coef_t/1e8:.2f}억원, p-value = {pval_t:.4e}')
print(f'      → 골목상권 대비 매출이 약 {coef_t/1e8:.2f}억원 높음')

---
## 6. 심층분석 시각화

### 6.1 표준화 회귀계수 비교

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

vars_plot = ['점포_수', '총_유동인구_수', '발달상권', '전통시장']
X_std = (X - X.mean()) / X.std()
X_std_const = sm.add_constant(X_std)
model_std = sm.OLS(y, X_std_const).fit()

std_coefs = [model_std.params[v] for v in vars_plot]
pvals = [model.pvalues[v] for v in vars_plot]
colors = ['#2ecc71' if p < 0.05 else '#e74c3c' for p in pvals]

bars = ax.barh(range(len(vars_plot)), std_coefs, color=colors, alpha=0.7)
ax.set_yticks(range(len(vars_plot)))
ax.set_yticklabels(['점포수', '유동인구', '발달상권', '전통시장'])
ax.set_xlabel('표준화 회귀계수 (Beta)')
ax.set_title('독립변수별 매출 영향력 비교\n(녹색: 유의함 p<0.05)')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(axis='x', alpha=0.3)

for i, (bar, pval) in enumerate(zip(bars, pvals)):
    width = bar.get_width()
    ax.text(width + 0.01, bar.get_y() + bar.get_height()/2, f'p={pval:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/07_회귀계수_영향력.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.2 잔차 분석

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

residuals = model.resid
fitted = model.fittedvalues

axes[0, 0].scatter(fitted/1e8, residuals/1e8, alpha=0.3, s=10)
axes[0, 0].axhline(y=0, color='red', linestyle='--')
axes[0, 0].set_xlabel('적합값 (억원)')
axes[0, 0].set_ylabel('잔차 (억원)')
axes[0, 0].set_title('잔차 vs 적합값')

axes[0, 1].hist(residuals/1e8, bins=50, edgecolor='white', alpha=0.7, color='steelblue')
axes[0, 1].set_xlabel('잔차 (억원)')
axes[0, 1].set_ylabel('빈도')
axes[0, 1].set_title('잔차 분포')

stats.probplot(residuals, plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot (정규성 검정)')

max_val = max(y.max(), fitted.max())/1e8
axes[1, 1].scatter(y/1e8, fitted/1e8, alpha=0.3, s=10)
axes[1, 1].plot([0, max_val], [0, max_val], 'r--', label='완벽한 예측선')
axes[1, 1].set_xlabel('실제 매출 (억원)')
axes[1, 1].set_ylabel('예측 매출 (억원)')
axes[1, 1].set_title('실제값 vs 예측값')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/08_잔차분석.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.3 VIF 시각화

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

colors_vif = ['#2ecc71' if v < 5 else '#f39c12' if v < 10 else '#e74c3c' for v in vif_data['VIF']]
bars = ax.bar(vif_data['변수'], vif_data['VIF'], color=colors_vif, alpha=0.7, width=0.5)

ax.axhline(y=5, color='orange', linestyle='--', label='주의 기준 (VIF=5)')
ax.axhline(y=10, color='red', linestyle='--', label='심각 기준 (VIF=10)')
ax.set_ylabel('VIF')
ax.set_title('다중공선성 검정 (VIF)\n녹색: VIF < 5 (문제없음)')
ax.legend()
ax.set_ylim(0, max(vif_data['VIF'].max() * 1.3, 6))

for bar, vif in zip(bars, vif_data['VIF']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, f'{vif:.2f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/09_VIF_다중공선성.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.4 상권유형별 평균 매출 비교

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

type_stats = merged.groupby('주요_상권유형')['당월_매출_금액'].agg(['mean', 'std', 'count']).reset_index()
type_stats['se'] = type_stats['std'] / np.sqrt(type_stats['count'])
type_stats = type_stats.sort_values('mean', ascending=False)

colors_type = ['#F28E2B', '#4E79A7', '#59A14F']
bars = ax.bar(type_stats['주요_상권유형'], type_stats['mean']/1e8, 
              yerr=type_stats['se']/1e8*1.96, capsize=5, color=colors_type, alpha=0.7)

ax.set_ylabel('평균 매출 (억원)')
ax.set_title('상권유형별 평균 매출 비교\n(95% 신뢰구간 표시)')
ax.grid(axis='y', alpha=0.3)

for bar, (idx, row) in zip(bars, type_stats.iterrows()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 3, f'{row["mean"]/1e8:.1f}억', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/10_상권유형_매출비교.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 7. 핵심 인사이트 요약

### 분석 결과 요약표

| 지표 | 값 | 해석 |
|------|-----|------|
| R² (설명력) | 0.6842 | 독립변수가 매출의 68.4% 설명 |
| 점포수 VIF | 2.86 | 다중공선성 없음 |
| 유동인구 VIF | 2.86 | 다중공선성 없음 |

### 변수별 영향력

| 변수 | 계수 | p-value | 해석 |
|------|------|---------|------|
| 점포수 | +4.11억원 | <0.001 | 1개 증가 시 4.11억 증가 |
| 유동인구 | +181원/명 | <0.001 | 100만명 당 1.81억 증가 |
| 발달상권 | +15.61억원 | <0.001 | 골목상권 대비 |
| 전통시장 | +11.45억원 | <0.001 | 골목상권 대비 |

### 핵심 인사이트
1. **점포수(밀집도)**가 매출에 가장 큰 영향을 미침
2. **유동인구**도 매출에 유의한 양(+)의 영향
3. **발달상권**은 골목상권 대비 가장 높은 매출 기대
4. 모든 독립변수가 통계적으로 **유의함** (p < 0.05)